In [ ]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [ ]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [ ]:
#stationär
data_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\data'
ab_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [49]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,81]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
dim_emb = 128
dim_hidden = 128
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 10


Vocabulary

In [ ]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


In [54]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
sequences = deepcopy(NCBI['AST_phenotypes'].tolist())
max_seq_len = [max_length[0],max_length[1]]
list_idx = []
list_SR = []
for i in range(len(sequences)):
    current_seq = sequences[i]
    current_idxs = []
    current_SRs = []
    for j in range(len(current_seq)):
        item = current_seq[j].split('=')
        abs = item[0]   
        sr = item[1]
        current_idxs.append(vocabulary_pheno.lookup_indices([abs]))
        for k in range(len(sr)):
            if sr == 'R':
                current_SRs.append(1)
            else:
                current_SRs.append(0)

    if len(current_idxs) != len(current_SRs):
        print("current sequence:",current_seq, "\n", "with length:", len(current_seq))
        print("indexes:",current_idxs, "with length:", len(current_idxs))
        print("suceptability",current_SRs, "with length:", len(current_SRs))
        print('error at', j)
        print("--------------------")
    current_idxs = [int(item[0]) for item in current_idxs]
    #for i in range(0,max_length[1] - len(current_idxs)):
    #    current_idxs.append(-1)
    #for i in range(0,max_length[1] - len(current_SRs)):
    #    current_SRs.append(-1)
    list_idx.append(current_idxs)
    list_SR.append(current_SRs)
for i in range(len(list_idx)):
    if len(list_idx[i]) != len(list_SR[i]):
        print('error at', i)

In [128]:
from torch.utils.data import DataLoader 
from bert_builder import BERT_ft
import torch.nn as nn
import copy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = torch.nn.BCEWithLogitsLoss()
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

optimizer = torch.optim.AdamW(bert_test.parameters(), lr=0.001, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)


loader = DataLoader(test_set, batch_size=16, shuffle=False)
for i, batch in enumerate(loader):
    if i >= 1:
        break 
    optimizer.zero_grad()
    input, token_target, attn_mask, AB_idx, SR_class = batch
    token_predictions, resistance_predictions = bert_test(input, attn_mask) 
    result_list = []
    for j in range(len(AB_idx)):
        result_tensor = torch.full((81,), -1)  # Create tensor filled with -1 values
        for idx, value in enumerate(AB_idx[j]):
            if value != -1:
                result_tensor[value.item()] = SR_class[j][idx]
                #print(AB_idx[j])
                #print(SR_class[j])
                #print(result_tensor)
        result_list.append(result_tensor)
    ab_loss = 0
    for i, row in enumerate(resistance_predictions):
        prediction = row
        print("pred",prediction, "shape ", prediction.shape)
        target = result_list[i]
        print("target",target, "shape ", target.shape)
        ab_loss =+ criterion(prediction, target) 
        print(ab_loss)
        
    #params_before_optimization = copy.deepcopy(bert_test.state_dict())

    # Call optimizer.step() to perform optimization
    #optimizer.step()

    # Compare parameter values after optimization
    #params_after_optimization = bert_test.state_dict()

    # Check if parameters were updated
    #parameters_updated = any((params_before_optimization[key] != params_after_optimization[key]).any() for key in params_before_optimization)

    # if parameters_updated:
    #     print("Parameters were updated after optimization.")
    # else:
    #     print("Parameters were not updated after optimization.")
    

target = torch.tensor([-1, -1, 0, 1, 0, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1,
                       -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1,
                       -1, -1, -1, 0, -1, -1, -1, 0, -1, -1, -1, 1, -1, -1, 0, -1, -1,
                       -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, 1, -1, -1, 0, 0, -1, -1, -1, -1,-1])
prediction = torch.tensor([0.3028, 0.8793, -0.0744, -0.5662, -0.2056, -0.5669, 0.4133, 0.5862,
                       0.1491, -0.1445, 0.2667, -0.4723, -0.2116, -0.9696, -0.0372, -0.6199,
                       0.0495, -0.6970, 0.8394, 0.3405, -0.2196, -0.7725, -0.2478, -0.1145,
                       0.1196, 0.0145, -0.1032, 0.6346, -0.2013, 1.4665, -1.6651, -0.5264,
                       1.7027, -0.1247, 0.7339, -0.6779, 0.0143, -0.2416, 0.6686, 0.4544,
                       0.5048, -0.8866, -0.5910, -0.7060, -0.9628, 0.2192, -0.8172, 0.3077,
                       0.4309, 0.1923, 0.0920, 0.0403, 0.2680, -0.5953, -0.3894, -0.0832,
                       -0.4327, 0.4032, -0.4555, -0.2467, -0.0788, -1.1045, -1.1388, -1.1328,
                       0.7328, -0.1226, -0.3339, -0.6213, -0.1118, 0.2419, -0.3614, -0.2195,
                       -0.1112, 0.3908, 0.4590, -0.4844, -0.0481, -0.1280, 1.2481, -0.3935,
                       -0.0614])

loss = loss_function(prediction, target.float())
print(loss)

pred tensor([-0.4827, -0.1044,  0.1782,  0.8553, -0.1718,  0.0107,  0.2306, -1.6142,
         0.1854,  0.3922, -0.3575,  0.1285,  0.1143,  0.8914,  0.2290, -0.0441,
         0.9951,  1.0961,  0.1067,  0.6577, -0.0310, -1.1902, -0.3373,  0.1316,
         0.0638,  0.8067, -0.3228, -0.1874,  0.3295,  0.4631, -0.4975, -0.4262,
         0.2297,  0.1139, -0.5967, -0.1866, -0.6816, -0.2082, -0.5836,  0.1944,
        -0.1727,  0.6549, -0.8431, -0.7730, -0.1036, -0.2091, -1.2821, -0.3943,
        -0.9584, -0.2915,  0.2416, -0.1794, -0.4063, -0.9677,  0.2807, -0.7922,
         1.5875, -0.3380,  0.5390, -0.5645,  0.2528, -0.0272, -0.0724,  1.7428,
        -0.3319, -0.6286, -0.1383, -0.3300,  0.3976, -0.1442,  0.0230, -0.1707,
         0.3696,  0.4899, -0.5966,  1.6022, -0.6710,  0.2210,  0.7581,  0.8992,
        -0.2489], grad_fn=<UnbindBackward0>) shape  torch.Size([81])
target tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1,

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [ ]:
list_AB_predictions = []
    for i, row in enumerate(resistance_predictions):
        AB_list = 0
        AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
        current_abs = []
        for ab in AB_list:
            current_abs.append(row[ab].item())
        current_abs = torch.tensor(current_abs, requires_grad=True)
        list_AB_predictions.append(current_abs)
    
    processed_tensor = [row[row != -1] for row in SR_class]
    ab_loss = 0
    for i, row in enumerate(processed_tensor):
        row = torch.tensor(row, dtype=torch.float32,requires_grad=True)
        list_AB_predictions[i] = torch.tensor(list_AB_predictions[i], dtype=torch.float32,requires_grad=True)

        ab_loss += self.ab_criterion(list_AB_predictions[i], row)

In [ ]:
test = res_pred[14]
print(test)
pred_res = torch.where(test > 0, torch.ones_like(test), torch.zeros_like(test))
print(pred_res)
indicies = [1,2,3,4]
test = pred_res[indicies]
print)

Trainer

In [ ]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [52]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import copy

class BertTrainer_ft:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.1
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        self.device = device
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.token_criterion = nn.CrossEntropyLoss(ignore_index = -1).to(self.device)
        self.ab_criterion = nn.BCEWithLogitsLoss().to(self.device)


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss_geno, avg_epoch_loss_ab = self.train(self.current_epoch)
            self.train_losses_geno.append(avg_epoch_loss_geno) 
            self.train_losses_ab.append(avg_epoch_loss_ab)  
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses_geno.append(val_results[0])
            self.val_losses_ab.append(val_results[1])
            self.val_accs.append(val_results[2])
            if self.wandb_mode:
                self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break
        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.best_epoch,
            "geno_train_losses": self.train_losses_geno,
            "ab_train_losses": self.train_losses_ab,
            "geno_val_losses": self.val_losses_geno,
            "ab_val_losses": self.val_losses_ab,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):

        self.train_losses_geno = []
        self.train_losses_ab = []

        self.val_losses_geno = []
        self.val_losses_ab = []

        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses_ab[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses_ab[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss_geno = 0
        epoch_loss_ab = 0

        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask, AB_idx, SR_class = batch
            
            ABinclusion = torch.unique(AB_idx)
            ABinclusion = ABinclusion[ABinclusion != -1]
            ABinclusion = ABinclusion.tolist()
            #self.model.exclude_networks(ABinclusion)

            self.optimizer.zero_grad() 

            token_predictions, resistance_predictions = self.model(input, attn_mask) 
            geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
            list_AB_predictions = []
            for i, row in enumerate(resistance_predictions):
                AB_list = 0
                AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                current_abs = []
                for ab in AB_list:
                    current_abs.append(row[ab].item())
                current_abs = torch.tensor(current_abs, requires_grad=True)
                list_AB_predictions.append(current_abs)
            
            processed_tensor = [row[row != -1] for row in SR_class]
            ab_loss = 0
            for i, row in enumerate(processed_tensor):
                row = torch.tensor(row, dtype=torch.float32,requires_grad=True)
                list_AB_predictions[i] = torch.tensor(list_AB_predictions[i], dtype=torch.float32,requires_grad=True)

                ab_loss += self.ab_criterion(list_AB_predictions[i], row)

            epoch_loss_geno += geno_loss.item()
            epoch_loss_ab += ab_loss.item()
            
            ab_loss.backward() 
            print("Optimizer state before step:", self.optimizer.state_dict())

            self.optimizer.step()
            #self.model.reset_exclusion()   
            
            print("Optimizer state after step:", self.optimizer.state_dict())    

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        return avg_epoch_loss_geno, avg_epoch_loss_ab
    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss_geno = 0
        epoch_loss_ab = 0
        total_correct = 0
        total_sum = 0
  
        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask, AB_idx, SR_class = batch

                token_predictions, resistance_predictions = self.model(input, attn_mask) 
                geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
                
                list_AB_predictions = []
                for i, row in enumerate(resistance_predictions):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    list_AB_predictions.append(current_abs)
                
                processed_tensor = [row[row != -1] for row in SR_class]
                ab_loss = 0
                for i, row in enumerate(processed_tensor):
                    row = row.type(torch.float32)
                    ab_loss += self.ab_criterion(list_AB_predictions[i], row)
                epoch_loss_geno += geno_loss.item()
                epoch_loss_ab += ab_loss.item() 
                
                list_AB_predictions = []
                for i, row in enumerate(resistance_predictions):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    current_abs = current_abs.type(torch.int16)
                    list_AB_predictions.append(current_abs)
                
                    processed_tensor = [row[row != -1] for row in SR_class]
                for i, row in enumerate(processed_tensor):
                    total_correct += (row == list_AB_predictions[i]).sum().item()
                    total_sum += len(row)

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        accuracy = total_correct / total_sum

        return avg_epoch_loss_geno, avg_epoch_loss_ab, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_embedding,
                'ff_dim': self.model.dim_embedding,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab_geno),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("GenoLosses/geno_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("GenoLosses/geno_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("AB_Losses/ab_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("AB_Losses/ab_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("Accuracies/val_acc", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            
            "GenoLosses/geno_train_loss": self.train_losses_geno[-1],
            "ABLosses/ab_train_loss": self.train_losses_ab[-1],

            "GenoLosses/geno_val_loss": self.val_losses_geno[-1],
            "ABLosses/ab_val_loss": self.val_losses_ab[-1],
            
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [ ]:
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset
include_pheno = False

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)


include_pheno = False
max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
from bert_builder import BERT_pt
from trainer import BertTrainer_pt

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERT_pt(vocab_size=len(vocabulary_geno), max_length=51, dim_embedding = 128, attention_heads=8, num_encoders=2, dropout_prob=0.2)
save_directory = 'c:\\Users\\erika\\Desktop\\Exjobb\\savefiles'

trainer = BertTrainer_pt(model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, save_directory)
trainer()

In [50]:
from bert_builder import BERT_ft
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr = 0.0001

include_pheno = True

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

reduced_samples = 1000
NCBI = NCBI.head(reduced_samples)

print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of token vocabulary:",len(vocabulary_geno))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)


Data loaded, 1000 samples found
length  of token vocabulary: 472


In [53]:
lr = 0.001
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

test = BertTrainer_ft(bert_test, train_set, val_set, 2, batch_size, lr, device, stop_patience,  False, "NCBI", "test")
test()

Epoch 1/2
Optimizer state before step: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.1, 'amsgrad': False, 'foreach': None, 'maximize': False, 'capturable': False, 'differentiable': False, 'fused': None, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]}]}
Optimizer state after step: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.1, 'amsgrad': False, 'foreach': None, 'maximize': False, 'capturab

{'best_epoch': 0,
 'geno_train_losses': [6.378661155700684, 6.375335330963135],
 'ab_train_losses': [24.38640480041504, 24.19566703796387],
 'geno_val_losses': [1.7904314994812012, 1.7904314994812012],
 'ab_val_losses': [6.035105495452881, 6.035105495452881],
 'val_accs': [0.584, 0.584]}

------------------------------

In [58]:
import torch

# Example tensors
AB_idx = torch.tensor([49, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1])

SR_class = torch.tensor([1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1])

# Create a new tensor filled with -1 values
result_tensor = torch.full_like(AB_idx, -1)

# Iterate over AB_idx and SR_class to place the values at correct indices
for idx, value in enumerate(AB_idx):
    if value != -1:
        result_tensor[value] = SR_class[idx]

print(result_tensor)

tensor([-1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1])
